<a href="https://colab.research.google.com/github/ganeshsundaresan/ganesund/blob/main/Dream_11_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ace_tools

In [ ]:
import pandas as pd
import itertools

# Updated 2025 squads with estimated credits and projected points
players = [
    # CSK Players
    {"name": "Rachin Ravindra", "team": "CSK", "role": "Batter", "credits": 9.0, "projected_points": 72},
    {"name": "Devon Conway", "team": "CSK", "role": "Batter", "credits": 9.0, "projected_points": 75},
    {"name": "Rahul Tripathi", "team": "CSK", "role": "Batter", "credits": 8.5, "projected_points": 65},
    {"name": "Vijay Shankar", "team": "CSK", "role": "Allrounder", "credits": 8.5, "projected_points": 64},
    {"name": "Shivam Dube", "team": "CSK", "role": "Allrounder", "credits": 8.5, "projected_points": 66},
    {"name": "MS Dhoni", "team": "CSK", "role": "Wicketkeeper", "credits": 8.0, "projected_points": 58},
    {"name": "Ravindra Jadeja", "team": "CSK", "role": "Allrounder", "credits": 9.5, "projected_points": 85},
    {"name": "Ravichandran Ashwin", "team": "CSK", "role": "Bowler", "credits": 8.5, "projected_points": 62},
    {"name": "Noor Ahmad", "team": "CSK", "role": "Bowler", "credits": 8.0, "projected_points": 60},
    {"name": "Anshul Kamboj", "team": "CSK", "role": "Bowler", "credits": 7.5, "projected_points": 50},
    {"name": "Khaleel Ahmed", "team": "CSK", "role": "Bowler", "credits": 8.0, "projected_points": 58},

    # LSG Players
    {"name": "Aiden Markram", "team": "LSG", "role": "Batter", "credits": 9.0, "projected_points": 78},
    {"name": "Nicholas Pooran", "team": "LSG", "role": "Wicketkeeper", "credits": 9.0, "projected_points": 74},
    {"name": "Rishabh Pant", "team": "LSG", "role": "Wicketkeeper", "credits": 9.5, "projected_points": 76},
    {"name": "Himmat Singh", "team": "LSG", "role": "Batter", "credits": 7.5, "projected_points": 52},
    {"name": "David Miller", "team": "LSG", "role": "Batter", "credits": 8.5, "projected_points": 68},
    {"name": "Abdul Samad", "team": "LSG", "role": "Allrounder", "credits": 8.0, "projected_points": 61},
    {"name": "Shardul Thakur", "team": "LSG", "role": "Allrounder", "credits": 8.5, "projected_points": 63},
    {"name": "Shahbaz Ahmed", "team": "LSG", "role": "Allrounder", "credits": 8.0, "projected_points": 60},
    {"name": "Akash Deep", "team": "LSG", "role": "Bowler", "credits": 7.5, "projected_points": 55},
    {"name": "Avesh Khan", "team": "LSG", "role": "Bowler", "credits": 8.0, "projected_points": 59},
    {"name": "Ravi Bishnoi", "team": "LSG", "role": "Bowler", "credits": 8.5, "projected_points": 67}
]

# Convert to DataFrame
df = pd.DataFrame(players)

# Compute a value metric = projected_points per credit
df["value"] = df["projected_points"] / df["credits"]

# Define minimum required players for each role
min_role_counts = {
    "Wicketkeeper": 1,
    "Batter": 3,
    "Allrounder": 1,
    "Bowler": 3
}

# Define maximum allowed players for each role
max_role_counts = {
    "Wicketkeeper": 4,
    "Batter": 6,
    "Allrounder": 4,
    "Bowler": 6
}

# Initialize selected list, along with trackers for credits and counts
selected = []
total_credits = 0
role_count = {"Wicketkeeper": 0, "Batter": 0, "Allrounder": 0, "Bowler": 0}
team_count = {}

# Step 1: For each role, select the minimal required players based on highest value
for role, required_count in min_role_counts.items():
    candidates = df[df["role"] == role].sort_values(by="value", ascending=False)
    count_added = 0
    for candidate in candidates.itertuples(index=False):
        if count_added >= required_count:
            break
        # Skip if already selected
        if candidate.name in [p["name"] for p in selected]:
            continue
        # Check credit constraint
        if total_credits + candidate.credits > 100:
            continue
        # Check team constraint (max 7 per team)
        current_team_count = team_count.get(candidate.team, 0)
        if current_team_count >= 7:
            continue
        # Accept candidate
        selected.append(candidate._asdict())
        total_credits += candidate.credits
        role_count[candidate.role] += 1
        team_count[candidate.team] = current_team_count + 1
        count_added += 1

# Step 2: Fill remaining slots (to reach 11) with overall best players by value,
# while ensuring we don't violate constraints.
remaining_slots = 11 - len(selected)
remaining_df = df[~df["name"].isin([p["name"] for p in selected])].sort_values(by="value", ascending=False)

for candidate in remaining_df.itertuples(index=False):
    if remaining_slots <= 0:
        break
    # Check if adding the candidate exceeds credit limit
    if total_credits + candidate.credits > 100:
        continue
    # Check if team limit is reached
    current_team_count = team_count.get(candidate.team, 0)
    if current_team_count >= 7:
        continue
    # Check if adding candidate violates maximum for their role
    if role_count[candidate.role] >= max_role_counts[candidate.role]:
        continue
    # If all good, add candidate
    selected.append(candidate._asdict())
    total_credits += candidate.credits
    role_count[candidate.role] += 1
    team_count[candidate.team] = current_team_count + 1
    remaining_slots -= 1

# Verify we have exactly 11 players
if len(selected) != 11:
    print("Could not form a valid team with the given constraints!")
else:
    # Convert selected list to DataFrame and sort by projected_points (highest first)
    selected_df = pd.DataFrame(selected).sort_values(by="projected_points", ascending=False).reset_index(drop=True)

    # Assign Captain and Vice-Captain: top two by projected_points become Captain & Vice-Captain
    selected_df.loc[0, "role_in_team"] = "Captain"
    selected_df.loc[1, "role_in_team"] = "Vice-Captain"
    selected_df.loc[2:, "role_in_team"] = "Player"

    # Display final team along with used credits, role counts, and team counts for verification:
    print("Total Credits Used:", total_credits)
    print("Role Counts:", role_count)
    print("Team Counts:", team_count)
    print("\nOptimal Dream11 Team (Greedy Approach):")
    print(selected_df)

    # If you wish to use ace_tools for displaying in a Streamlit-like UI:
    try:
        import ace_tools as tools
        tools.display_dataframe_to_user(name="Dream11 Team (CSK vs LSG - 2025)", dataframe=selected_df)
    except ImportError:
        pass  # otherwise, we already printed the DataFrame


Total Credits Used: 96.5
Role Counts: {'Wicketkeeper': 2, 'Batter': 4, 'Allrounder': 2, 'Bowler': 3}
Team Counts: {'LSG': 6, 'CSK': 5}

Optimal Dream11 Team (Greedy Approach):
               name team          role  credits  projected_points     value  \
0   Ravindra Jadeja  CSK    Allrounder      9.5                85  8.947368   
1     Aiden Markram  LSG        Batter      9.0                78  8.666667   
2      Rishabh Pant  LSG  Wicketkeeper      9.5                76  8.000000   
3      Devon Conway  CSK        Batter      9.0                75  8.333333   
4   Nicholas Pooran  LSG  Wicketkeeper      9.0                74  8.222222   
5   Rachin Ravindra  CSK        Batter      9.0                72  8.000000   
6      David Miller  LSG        Batter      8.5                68  8.000000   
7      Ravi Bishnoi  LSG        Bowler      8.5                67  7.882353   
8       Shivam Dube  CSK    Allrounder      8.5                66  7.764706   
9        Noor Ahmad  CSK        Bo